In [1]:
import pandas as pd
import math
from scipy.interpolate import interp1d
from scipy.optimize import newton
from dateutil.relativedelta import relativedelta

def curva(plazos, dfs):
    """
    Esta función permite obtener una curva de interpolación, es decir, entrega una función que permite que en cualquier momento uno obtenga el punto de una respectiva
    curva dado una metodo de interpolación log-lineal.
    :param plazos: Lista que contiene los plazos de cada uno de los puntos de la curva.
    :param dfs: Lista que contiene los factores de descuento, o tasas, que corresponden a cada uno de los puntos de la curva.
    """
    def curva_interp(plazo):
        log_dfs = []
        for df in dfs:
            log = math.log(df)
            log_dfs.append(log)
        return math.exp(interp1d(plazos, log_dfs, kind='linear', fill_value = 'extrapolate')(plazo))
        #return interp1d(plazos, dfs, kind='cubic', fill_value = 'extrapolate')(plazo)
    return curva_interp

def ten(tenor):
    """
    Esta función sirve para determinar la cantidad de meses que tiene un tenor dado.
    :param tenor: str que contiene el tenor de una determinada tasa 
    """
    symbol = tenor[-1:].lower() 
    if symbol == "m":
        result = int(tenor[:-1])
    elif symbol == "y":
        result = 12 * int(tenor[:-1])
    elif symbol == "w":
        result = float(tenor[:-1])/4
    else:
        result = 0
    return result

def weeekend_dias(fecha,signo):
    """
    Esta función permite avanzar una fecha dada cuando nos encontremos en fin de semana.
    :param fecha: Fecha que quiere moverse.
    :param signo: Sentido en el cual se quiere mover la fecha. 'positivo' mueve la fecha en hacia adelante y 'negativo' mueve la fecha hacia atras cuando nos encontramos a fin de mes.
    """
    mov_dia = {'positivo':{'sabado': 2, 'domingo': 1}, 'negativo':{'sabado': -1, 'domingo': -2}}
    diasemana = fecha.weekday()
    if diasemana == 5: #dia Sabado
        dias = timedelta(days=mov_dia[signo]['sabado']) # diferencia de tiempo de 2 dias para sumarlos a la fecha en cuestión 
    elif diasemana == 6: #Dia domingo
        dias = timedelta(days=mov_dia[signo]['domingo'])
    else:
        dias = timedelta(days=0)
    return fecha + dias

def busday(fecha, sentido="MOD_FOLLOW"):
    """
    Esta función permite avanzar en sentido MOD_FOLLOW una fecha dada.
    :param fecha: Fecha que se quiere avanzar en sentido MOD_FOLLOW
    """
    fecha_nueva = weeekend_dias(fecha,'positivo')
    if fecha_nueva.month > fecha.month:
        fecha_nueva =  weeekend_dias(fecha,'negativo')
    return fecha_nueva

def addmonth (fecha, numeses):
    """
    Esta función permite sumar una cantidad determinada de meses a una fecha dada.
    :param fecha: Fecha a la cual se le quiere sumas cierta cantidad de meses.
    :param numeses: Número de meses que serán sumados.
    """
    meses = relativedelta(months=numeses)
    nuevo = fecha + meses
    return nuevo

## valorizacion inicial:

In [2]:
#pata fija
from datetime import date, timedelta
fecha_inicial = date(2019,10,8)
madurez = '2Y'
periodicidad = '6M'
num_cupones = 4
nominal = 1000000.0
tasa_2y = 0.018
curva_ois = [[1.0, 30.0, 90.0, 180.0, 360.0, 540.0, 720.0], [0.99, 0.98, 0.97, 0.96, 0.94, 0.92, 0.90]]
curva_int_ois = curva(curva_ois[0], curva_ois[1])

In [3]:
curva_int_ois(15.0)

0.9851597377061232

### Ejemplo valorización swap 2y:

In [4]:
# pata fija -> devengo de intereses lineal actual 360
def swap_pata_fija(fecha_inicial, madurez, periodicidad, num_cupones, nominal, tasa_2y, curva_int_ois):
    swap_pata_fija = []
    fecha_ini = fecha_inicial
    for i in range(1,num_cupones+1):
    
        fecha_fin = busday(addmonth(fecha_inicial, i*ten(periodicidad)))
        dias_intereses = float((fecha_fin - fecha_ini).days)
        dias_desde_inicio = float((fecha_fin - fecha_inicial).days)
        interes = nominal*tasa_2y*dias_intereses/360.0
        df = curva_int_ois(dias_desde_inicio)
    
        if i == num_cupones:
            valor_presente = (nominal + interes)*df
        else:
            valor_presente = interes*df
    
        swap_pata_fija.append((fecha_ini, fecha_fin, dias_intereses, dias_desde_inicio, interes, df, valor_presente))
        fecha_ini = fecha_fin
    
    swap_pata_fija = pd.DataFrame(swap_pata_fija, columns = ["fecha_ini", "fecha_fin", "dias_intereses", "dias_desde_inicio", "interes", "df", "valor_presente"]) 
    return swap_pata_fija

In [5]:
pata_fija = swap_pata_fija(fecha_inicial, madurez, periodicidad, num_cupones, nominal, tasa_2y, curva_int_ois)
pata_fija

,fecha_ini,fecha_fin,dias_intereses,dias_desde_inicio,interes,df,valor_presente
0,2019-10-08,2020-04-08,183.0,183.0,9150.0,0.959663,8780.918322
1,2020-04-08,2020-10-08,183.0,366.0,9150.0,0.939326,8594.836380
2,2020-10-08,2021-04-08,182.0,548.0,9100.0,0.919102,8363.825886
3,2021-04-08,2021-10-08,183.0,731.0,9150.0,0.898792,907015.918137


In [6]:
def curva_tasas(plazos, tasas):
    """
    Esta función permite obtener una curva de interpolación, es decir, entrega una función que permite que en cualquier momento uno obtenga el punto de una respectiva
    curva dado una metodo de interpolación lineal.
    :param plazos: Lista que contiene los plazos de cada uno de los puntos de la curva.
    :param dfs: Lista que contiene los factores de descuento, o tasas, que corresponden a cada uno de los puntos de la curva.
    """
    def curva_interp(plazo):
        return interp1d(plazos, tasas, kind='linear', fill_value = 'extrapolate')(plazo)
    return curva_interp

In [7]:
# Pata Flotante -> devengo de intereses es linal 30/360

periodicidad = '3M'
num_cupones = 8
libor_inicial = 0.02
curva_libor_3m = [[1.0, 30.0, 90.0, 180.0, 360.0, 540.0, 720.0], [0.02, 0.0199, 0.0196, 0.0194, 0.0190, 0.0187, 0.0180]]
curva_int_libor_3m = curva_tasas(curva_libor_3m[0], curva_libor_3m[1])

In [8]:
def swap_pata_flotante(fecha_inicial, madurez, periodicidad, num_cupones, nominal, libor_inicial, curva_int_libor_3m, curva_int_ois):
    swap_pata_flotante = []
    fecha_ini = fecha_inicial

    for i in range(1,num_cupones+1):
    
        fecha_fin = busday(addmonth(fecha_inicial, i*ten(periodicidad)))
        dias_intereses = 30.0*ten(periodicidad)
        dias_desde_inicio = float((fecha_fin - fecha_inicial).days)
    
        if i == 1:
            tasa_fwd = libor_inicial
            interes = nominal*libor_inicial*dias_intereses/360.0
        else:
            df_ini = 1.0 /(1.0 + curva_int_libor_3m(dias_desde_inicio_ini)*dias_desde_inicio_ini/360.0)
            df_fin = 1.0 /(1.0 + curva_int_libor_3m(dias_desde_inicio)*dias_desde_inicio/360.0)
            tasa_fwd = (df_ini/df_fin - 1.0)*360.0/(dias_desde_inicio - dias_desde_inicio_ini)
            interes = nominal*tasa_fwd*dias_intereses/360.0
        
        df = curva_int_ois(dias_desde_inicio)
    
        if i == num_cupones:
            valor_presente = (nominal + interes)*df
        else:
            valor_presente = interes*df
    
        swap_pata_flotante.append((fecha_ini, fecha_fin, dias_intereses, dias_desde_inicio,tasa_fwd, interes, df, valor_presente))
        fecha_ini = fecha_fin
        dias_desde_inicio_ini = dias_desde_inicio
    
    swap_pata_flotante = pd.DataFrame(swap_pata_flotante, columns = ["fecha_ini", "fecha_fin", "dias_intereses", "dias_desde_inicio", "tasa_fwd","interes", "df", "valor_presente"])
    return swap_pata_flotante

In [9]:
pata_flot = swap_pata_flotante(fecha_inicial, madurez, periodicidad, num_cupones, nominal, libor_inicial, curva_int_libor_3m, curva_int_ois)
pata_flot

,fecha_ini,fecha_fin,dias_intereses,dias_desde_inicio,tasa_fwd,interes,df,valor_presente
0,2019-10-08,2020-01-08,90.0,92.0,0.020000,5000.000000,0.969777,4848.883250
1,2020-01-08,2020-04-08,90.0,183.0,0.019093,4773.318621,0.959663,4580.778244
2,2020-04-08,2020-07-08,90.0,274.0,0.018601,4650.267483,0.949503,4415.443087
3,2020-07-08,2020-10-08,90.0,366.0,0.018126,4531.568992,0.939326,4256.622298
4,2020-10-08,2021-01-08,90.0,458.0,0.017881,4470.359668,0.929058,4153.222464
5,2021-01-08,2021-04-08,90.0,548.0,0.017398,4349.537437,0.919102,3997.667452
6,2021-04-08,2021-07-08,90.0,639.0,0.015737,3934.170196,0.908946,3575.946779
7,2021-07-08,2021-10-08,90.0,731.0,0.014985,3746.267672,0.898792,902159.086904


In [10]:
#recibe fija
#paga flotante

valor_swap = pata_fija['valor_presente'].sum() - pata_flot['valor_presente'].sum()
valor_swap

767.8482471448369

### Como encontrar la tasa 2Y:

In [11]:
# Hacemos una función que dependa de la tasa desconocida:
def valor_swap(pata_fija, pata_flotante, curva_proyeccion, curva_descuento, x):
    def discount(x):
        curva_proyeccion[1][-1] = x
        curva_proyeccion = curva_tasas(curva_proyeccion[0],curva_proyeccion[1])
        curva_descuento = curva(curva_descuento[0],curva_descuento[1])
        fix = swap_pata_fija(swap_pata_fija[0],swap_pata_fija[1],swap_pata_fija[2],swap_pata_fija[3],swap_pata_fija[4],swap_pata_fija[5],curva_descuento)
        flot = swap_pata_flotante(pata_flotante[0],pata_flotante[1],pata_flotante[2],pata_flotante[3],pata_flotante[4],pata_flotante[5],curva_proyeccion,curva_descuento)
        vp = 100000.0*(fix['valor_presente'].sum()-flot['valor_presente'].sum())
        return vp
    return discount

In [12]:
pata_fija = [fecha_inicial, madurez, periodicidad, num_cupones, nominal, tasa_2y]
pata_flotante = [fecha_inicial, madurez, periodicidad, num_cupones, nominal, libor_inicial]
r = 0.9
vp = valor_swap(pata_fija, pata_flotante, curva_libor_3m, curva_ois, r)
Rate = newton(r,vp)

TypeError: unsupported operand type(s) for *: 'float' and 'function'